# Python String Methods for Data Engineering

This notebook covers essential string methods used in Data Engineering for:
- Data cleaning and normalization
- Parsing log files, CSV data, and structured text
- Validating and transforming data fields

**Categories covered:**
1. Cleaning & Normalization
2. Case Transformation
3. Splitting & Joining
4. Search & Find
5. Validation & Checking
6. Formatting & Alignment

---
# Section 1: Cleaning & Normalization

The most common task in Data Engineering - cleaning messy data!

## 1.1 `strip()` - Remove Whitespace from Both Ends

**What it does:** Removes leading and trailing whitespace (or specified characters)

**Syntax:**
```
string.strip([chars])
       ↓        ↓
       |        └── Optional: specific characters to remove
       └── The string to clean
```

**DE Use Case:** Raw data from CSV files, databases, or APIs often has extra spaces

In [ ]:
# Sample: Raw customer data from a CSV file
raw_records = [
    "   John Doe   ",
    "Jane Smith  ",
    "  Bob Johnson",
    "Alice Brown"
]

print("Before strip():")
print("-" * 30)
for record in raw_records:
    print(f"'{record}'  (length: {len(record)})")

In [ ]:
# Apply strip() to clean the data
cleaned_records = [record.strip() for record in raw_records]

print("After strip():")
print("-" * 30)
for record in cleaned_records:
    print(f"'{record}'  (length: {len(record)})")

In [ ]:
# strip() with specific characters
# Common in DE: removing delimiters, quotes, or special chars

raw_ids = [
    "###ID_001###",
    "---ID_002---",
    "***ID_003***"
]

print("Removing specific characters:")
print("-" * 40)
for raw_id in raw_ids:
    # strip() removes ANY of the characters in the string, not the whole string
    cleaned = raw_id.strip("#-*")
    print(f"'{raw_id}' → '{cleaned}'")

## 1.2 `lstrip()` and `rstrip()` - Remove from One Side Only

**Syntax:**
```
string.lstrip([chars])  # Left side only
string.rstrip([chars])  # Right side only
```

**DE Use Cases:**
- `lstrip('0')`: Remove leading zeros from IDs
- `rstrip('\n')`: Remove trailing newlines from file data

In [ ]:
# lstrip() - Remove leading zeros from IDs
# Common scenario: ID fields stored with padding

padded_ids = ["000042", "000001", "001234", "000000"]

print("Removing leading zeros:")
print("-" * 30)
for pid in padded_ids:
    cleaned = pid.lstrip('0')
    # Handle edge case: if all zeros, keep at least one
    cleaned = cleaned if cleaned else '0'
    print(f"'{pid}' → '{cleaned}'")

In [ ]:
# rstrip() - Remove trailing newlines from file data
# Common when reading lines from files

file_lines = [
    "data_value_1\n",
    "data_value_2\n\n",
    "data_value_3\r\n"
]

print("Removing trailing newlines:")
print("-" * 40)
for line in file_lines:
    cleaned = line.rstrip('\n\r')
    print(f"{repr(line):25} → '{cleaned}'")

## 1.3 `replace()` - Replace Substrings

**What it does:** Replaces all (or limited) occurrences of a substring

**Syntax:**
```
string.replace(old, new[, count])
               ↓    ↓      ↓
               |    |      └── Optional: max replacements
               |    └── What to replace with
               └── What to find and replace
```

**DE Use Cases:**
- Standardizing phone numbers, dates
- Fixing inconsistent delimiters
- Data masking

In [ ]:
# DE Use Case: Standardizing phone numbers
# Different sources have different formats

raw_phones = [
    "123-456-7890",
    "(123) 456-7890",
    "123.456.7890",
    "123 456 7890"
]

print("Standardizing phone numbers to digits only:")
print("-" * 40)
for phone in raw_phones:
    # Chain multiple replace() calls
    cleaned = phone.replace("-", "").replace("(", "").replace(")", "").replace(" ", "").replace(".", "")
    print(f"'{phone:20}' → '{cleaned}'")

In [ ]:
# replace() with count parameter
# Useful when you only want to replace first N occurrences

log_line = "ERROR: Connection ERROR in module ERROR_HANDLER"

print(f"Original: {log_line}")
print(f"Replace all:    {log_line.replace('ERROR', 'WARN')}")
print(f"Replace first:  {log_line.replace('ERROR', 'WARN', 1)}")
print(f"Replace first 2: {log_line.replace('ERROR', 'WARN', 2)}")

## 1.4 `removeprefix()` and `removesuffix()` (Python 3.9+)

**What they do:** Remove a specific prefix or suffix if present

**Syntax:**
```
string.removeprefix(prefix)  # Remove from start
string.removesuffix(suffix)  # Remove from end
```

**Important:** Unlike `strip()`, these remove the EXACT string, not individual characters

**DE Use Cases:**
- Removing consistent prefixes from IDs (ORDER_, TXN_, etc.)
- Removing file extensions

In [ ]:
# removeprefix() - Clean prefixes from IDs
order_ids = [
    "ORD_12345",
    "ORD_67890",
    "TXN_11111",  # Different prefix - won't be affected
    "ORD_99999"
]

print("Removing 'ORD_' prefix:")
print("-" * 30)
for oid in order_ids:
    cleaned = oid.removeprefix("ORD_")
    print(f"'{oid}' → '{cleaned}'")

In [ ]:
# removesuffix() - Remove file extensions
filenames = [
    "report_2024.csv",
    "data_export.csv",
    "summary.json",  # Different extension - won't be affected
    "transactions.csv"
]

print("Removing '.csv' suffix:")
print("-" * 30)
for fname in filenames:
    cleaned = fname.removesuffix(".csv")
    print(f"'{fname}' → '{cleaned}'")

### strip() vs removeprefix()/removesuffix() - Key Difference!

This is a common source of confusion:

In [ ]:
# IMPORTANT: Understanding the difference

text = "aaahelloaaa"

# strip('aaa') removes individual 'a' characters from both ends
# NOT the string 'aaa' as a unit
print(f"strip('aaa'):        '{text}' → '{text.strip('aaa')}'")

# removeprefix/removesuffix removes the EXACT string
print(f"removeprefix('aaa'): '{text}' → '{text.removeprefix('aaa')}'")
print(f"removesuffix('aaa'): '{text}' → '{text.removesuffix('aaa')}'")

print("\n" + "="*50)
print("strip() = remove any of these CHARACTERS")
print("removeprefix() = remove this exact STRING")

---
# Section 2: Case Transformation

Essential for data normalization and standardization

## 2.1 `lower()` and `upper()` - Case Conversion

**Syntax:**
```
string.lower()  # Convert all to lowercase
string.upper()  # Convert all to uppercase
```

**DE Use Cases:**
- Normalizing emails (always lowercase)
- Case-insensitive comparisons
- Standardizing codes/identifiers

In [ ]:
# DE Use Case: Normalizing email addresses
# Emails should always be stored in lowercase for consistency

raw_emails = [
    "John.Doe@Company.COM",
    "JANE.SMITH@EXAMPLE.ORG",
    "Bob_Jones@Test.Net",
    "alice@domain.com"
]

print("Normalizing emails to lowercase:")
print("-" * 50)
for email in raw_emails:
    normalized = email.lower()
    print(f"{email:30} → {normalized}")

In [ ]:
# DE Use Case: Standardizing country codes
# Codes should be uppercase for consistency

country_codes = ["us", "Gb", "DE", "fr", "JP"]

print("Standardizing country codes to uppercase:")
print("-" * 30)
for code in country_codes:
    standardized = code.upper()
    print(f"'{code}' → '{standardized}'")

In [ ]:
# DE Use Case: Case-insensitive comparison
# When matching records from different sources

source_a = "John Doe"
source_b = "JOHN DOE"

# Direct comparison fails
print(f"Direct comparison: '{source_a}' == '{source_b}' → {source_a == source_b}")

# Normalize before comparing
print(f"Normalized:        '{source_a.lower()}' == '{source_b.lower()}' → {source_a.lower() == source_b.lower()}")

## 2.2 `title()` and `capitalize()` - Proper Formatting

**Syntax:**
```
string.title()       # Capitalize First Letter Of Each Word
string.capitalize()  # Capitalize only first letter of string
```

**DE Use Cases:**
- Formatting names for display
- Standardizing city/country names

In [ ]:
# Formatting customer names
raw_names = [
    "john doe",
    "JANE SMITH",
    "bOB jOHNSON",
    "alice brown"
]

print("Formatting names with title():")
print("-" * 30)
for name in raw_names:
    formatted = name.title()
    print(f"'{name:15}' → '{formatted}'")

In [ ]:
# title() vs capitalize()
text = "hello world from python"

print(f"Original:    '{text}'")
print(f"title():     '{text.title()}'")
print(f"capitalize(): '{text.capitalize()}'")

---
# Section 3: Splitting & Joining

Critical for parsing structured data (CSV, logs, paths)

## 3.1 `split()` - Break String into List

**What it does:** Splits a string into a list using a delimiter

**Syntax:**
```
string.split([sep[, maxsplit]])
              ↓       ↓
              |       └── Optional: max number of splits
              └── Delimiter (default: whitespace)
```

**Returns:** A list of substrings

**DE Use Cases:**
- Parsing CSV rows
- Extracting fields from log lines
- Breaking apart paths

In [ ]:
# Basic split() - parsing CSV-like data
csv_row = "John,Doe,30,Engineer,New York"

# Split on comma
fields = csv_row.split(",")

print(f"Original: '{csv_row}'")
print(f"Split:    {fields}")
print(f"Type:     {type(fields)}")
print()
print("Accessing individual fields:")
print(f"  fields[0] (first name): {fields[0]}")
print(f"  fields[1] (last name):  {fields[1]}")
print(f"  fields[2] (age):        {fields[2]}")

In [ ]:
# split() with maxsplit - Parsing log lines
# Common pattern: split date/time, then keep message together

log_line = "2024-01-15 10:30:45 ERROR Connection failed: timeout after 30s"

# Without maxsplit - splits everything
all_parts = log_line.split(" ")
print(f"split(' '):     {all_parts}")
print(f"  → {len(all_parts)} parts (message is fragmented!)")
print()

# With maxsplit=3 - keeps message together
parts = log_line.split(" ", 3)
print(f"split(' ', 3):  {parts}")
print(f"  → Date: {parts[0]}")
print(f"  → Time: {parts[1]}")
print(f"  → Level: {parts[2]}")
print(f"  → Message: {parts[3]}")

In [ ]:
# split() with no argument - splits on whitespace
# Handles multiple spaces, tabs, newlines automatically

messy_data = "John    Doe\t\t30   Engineer"

print(f"Original: {repr(messy_data)}")
print(f"split():  {messy_data.split()}")
print()
print("Note: split() with no args handles multiple whitespace cleanly!")

## 3.2 `rsplit()` - Split from Right Side

**Syntax:** Same as `split()`, but splits from the right

**DE Use Case:** When you want the LAST N parts (e.g., file path → filename)

In [ ]:
# rsplit() - Extract filename from path
file_path = "/home/user/data/projects/report_2024.csv"

# Get just the filename (last part)
parts = file_path.rsplit("/", 1)
print(f"Path: {file_path}")
print(f"rsplit('/', 1): {parts}")
print(f"  → Directory: {parts[0]}")
print(f"  → Filename:  {parts[1]}")

In [ ]:
# split() vs rsplit() comparison
text = "a.b.c.d.e"

print(f"Original: {text}")
print(f"split('.', 2):  {text.split('.', 2)}   ← splits from LEFT")
print(f"rsplit('.', 2): {text.rsplit('.', 2)}  ← splits from RIGHT")

## 3.3 `join()` - Combine List into String

**What it does:** The opposite of `split()` - joins list elements with a separator

**Syntax:**
```
separator.join(iterable)
    ↓            ↓
    |            └── List/tuple of strings to join
    └── String to put between elements
```

**Note:** The separator calls `.join()`, not the list!

**DE Use Cases:**
- Building CSV rows
- Creating file paths
- Constructing queries

In [ ]:
# join() - Building CSV rows
fields = ["John", "Doe", "30", "Engineer"]

# Join with comma
csv_row = ",".join(fields)

print(f"List:    {fields}")
print(f"Joined:  '{csv_row}'")
print()

# Join with different separators
print(f"Tab-separated:   '{chr(9).join(fields)}'")
print(f"Pipe-separated:  '{'|'.join(fields)}'")

In [ ]:
# join() - Building file paths
path_parts = ["home", "user", "data", "file.csv"]

unix_path = "/".join(path_parts)
windows_path = "\\".join(path_parts)

print(f"Parts:        {path_parts}")
print(f"Unix path:    {unix_path}")
print(f"Windows path: {windows_path}")

In [ ]:
# IMPORTANT: join() only works with strings!
# If you have numbers, convert them first

numbers = [1, 2, 3, 4, 5]

# This will fail:
# ",".join(numbers)  # TypeError!

# Convert to strings first
number_strings = [str(n) for n in numbers]
result = ",".join(number_strings)

print(f"Numbers: {numbers}")
print(f"Joined:  '{result}'")
print()

# One-liner version:
result_oneliner = ",".join(str(n) for n in numbers)
print(f"One-liner: '{result_oneliner}'")

## 3.4 `partition()` - Split into Three Parts

**What it does:** Splits on FIRST occurrence, returns tuple of (before, separator, after)

**Syntax:**
```
string.partition(sep)
→ Returns: (before_sep, sep, after_sep)
```

**DE Use Case:** Extracting key-value pairs, splitting on first delimiter

In [ ]:
# partition() - Parsing key-value pairs
# Common in config files, query strings

config_lines = [
    "database_host=localhost",
    "connection_string=host=db;port=5432;user=admin",  # Value contains '='!
    "timeout=30"
]

print("Parsing config with partition():")
print("-" * 50)
for line in config_lines:
    key, sep, value = line.partition("=")
    print(f"Key: '{key:20}' | Value: '{value}'")

In [ ]:
# Why partition() is better than split() for key-value:

line = "connection_string=host=db;port=5432"

# split() breaks on ALL '=' characters
split_result = line.split("=")
print(f"split('='):     {split_result}")
print(f"  → Value is fragmented!")
print()

# partition() only splits on FIRST '='
key, _, value = line.partition("=")
print(f"partition('='): ('{key}', '=', '{value}')")
print(f"  → Value stays intact!")

---
# Section 4: Search & Find

Locating substrings within data

## 4.1 `find()` and `rfind()` - Locate Substrings

**Syntax:**
```
string.find(sub[, start[, end]])   # Search from left
string.rfind(sub[, start[, end]])  # Search from right
```

**Returns:** Index of first/last occurrence, or **-1 if not found**

**DE Use Cases:**
- Checking if substring exists
- Locating delimiters for parsing
- Extracting parts of strings

In [ ]:
# find() - Basic usage
log = "ERROR: Connection failed at 10:30:45"

# Find returns index position
error_pos = log.find("ERROR")
warning_pos = log.find("WARNING")

print(f"Log: '{log}'")
print(f"find('ERROR'):   {error_pos}  (found at index 0)")
print(f"find('WARNING'): {warning_pos}  (-1 means not found)")
print()

# Common pattern: check if substring exists
if log.find("ERROR") != -1:
    print("This is an error log!")

In [ ]:
# rfind() - Find from right (last occurrence)
path = "/home/user/data/archive/file.csv"

# Find last slash to get filename
last_slash = path.rfind("/")
filename = path[last_slash + 1:]

print(f"Path: {path}")
print(f"Last '/' at index: {last_slash}")
print(f"Filename: {filename}")

In [ ]:
# find() vs 'in' operator
text = "Hello World"

# 'in' operator - just checks existence (True/False)
print(f"'World' in text: {'World' in text}")

# find() - gives you the position
print(f"text.find('World'): {text.find('World')}")

print("\nUse 'in' when you just need True/False")
print("Use find() when you need the position")

## 4.2 `count()` - Count Occurrences

**Syntax:**
```
string.count(sub[, start[, end]])
```

**Returns:** Number of non-overlapping occurrences

**DE Use Cases:**
- Validating data format (e.g., email has exactly one @)
- Counting delimiters to verify field count

In [ ]:
# count() - Data validation example
emails = [
    "john@company.com",
    "invalid.email.com",    # No @
    "bad@@email.com",        # Two @
    "jane@dept@company.com"  # Two @
]

print("Email validation using count():")
print("-" * 50)
for email in emails:
    at_count = email.count("@")
    is_valid = at_count == 1
    print(f"{email:25} | @count: {at_count} | valid: {is_valid}")

In [ ]:
# count() - Verify CSV field count
csv_rows = [
    "John,Doe,30,Engineer",
    "Jane,Smith,25",              # Missing field
    "Bob,Johnson,40,Manager,NYC"  # Extra field
]

expected_fields = 4  # We expect 4 fields (3 commas)

print("Validating CSV row structure:")
print("-" * 50)
for row in csv_rows:
    comma_count = row.count(",")
    actual_fields = comma_count + 1
    is_valid = actual_fields == expected_fields
    print(f"{row:30} | fields: {actual_fields} | valid: {is_valid}")

---
# Section 5: Validation & Checking

Methods that return `True` or `False` - perfect for data validation

## 5.1 `startswith()` and `endswith()` - Check Prefix/Suffix

**Syntax:**
```
string.startswith(prefix)   # Check beginning
string.endswith(suffix)     # Check ending

# Can also check multiple options with tuple:
string.startswith(("opt1", "opt2", "opt3"))
```

**DE Use Cases:**
- Filtering files by extension
- Identifying record types by prefix
- Validating formats

In [ ]:
# startswith() - Filtering by prefix
transaction_ids = [
    "ORD_12345",
    "TXN_67890",
    "REF_11111",
    "ORD_22222",
    "TXN_33333"
]

# Filter only orders
orders = [tid for tid in transaction_ids if tid.startswith("ORD_")]
print(f"All transactions: {transaction_ids}")
print(f"Only orders:      {orders}")

In [ ]:
# endswith() with tuple - Check multiple extensions
files = [
    "report.csv",
    "data.json",
    "image.png",
    "config.yaml",
    "archive.parquet"
]

# Filter data files (csv, json, parquet)
data_extensions = (".csv", ".json", ".parquet")
data_files = [f for f in files if f.endswith(data_extensions)]

print(f"All files:  {files}")
print(f"Data files: {data_files}")

## 5.2 `isdigit()`, `isalpha()`, `isalnum()` - Character Type Checks

**Syntax:**
```
string.isdigit()  # True if all characters are digits (0-9)
string.isalpha()  # True if all characters are letters (a-z, A-Z)
string.isalnum()  # True if all characters are alphanumeric
```

**DE Use Cases:**
- Validating ID fields
- Checking numeric strings before conversion
- Data quality checks

In [ ]:
# isdigit() - Validate before converting to int
user_inputs = ["123", "45.6", "abc", "789", "-10", "100"]

print("Validating numeric inputs:")
print("-" * 40)
for inp in user_inputs:
    is_valid = inp.isdigit()
    print(f"'{inp:6}' isdigit(): {is_valid}")

print("\nNote: isdigit() is strict - no decimals, no negatives!")

In [ ]:
# isalnum() - Validate usernames/IDs
usernames = [
    "user123",
    "john_doe",     # underscore is not alphanumeric
    "jane.smith",   # dot is not alphanumeric
    "bobJohnson",
    "user@test"     # @ is not alphanumeric
]

print("Validating usernames (alphanumeric only):")
print("-" * 40)
for username in usernames:
    is_valid = username.isalnum()
    print(f"'{username:12}' isalnum(): {is_valid}")

In [ ]:
# isalpha() - Validate name fields
names = ["John", "Jane123", "O'Brien", "María", "Bob Smith"]

print("Validating names (letters only):")
print("-" * 40)
for name in names:
    is_valid = name.isalpha()
    print(f"'{name:12}' isalpha(): {is_valid}")

print("\nNote: spaces, apostrophes, etc. make isalpha() return False!")

## 5.3 `isspace()` - Check for Whitespace

**DE Use Case:** Detecting empty or whitespace-only fields (data quality)

In [ ]:
# Detecting empty/whitespace fields in data
records = [
    {"name": "John", "city": "NYC"},
    {"name": "Jane", "city": "   "},      # Whitespace only
    {"name": "Bob", "city": ""},           # Empty
    {"name": "   ", "city": "Chicago"},   # Whitespace name
]

print("Detecting problematic fields:")
print("-" * 50)
for i, record in enumerate(records):
    name_issue = not record["name"] or record["name"].isspace()
    city_issue = not record["city"] or record["city"].isspace()
    
    issues = []
    if name_issue:
        issues.append("name")
    if city_issue:
        issues.append("city")
    
    status = f"Issues: {issues}" if issues else "OK"
    print(f"Record {i}: {record} → {status}")

---
# Section 6: Formatting & Alignment

Creating formatted output and fixed-width fields

## 6.1 `zfill()` - Zero-Padding

**Syntax:**
```
string.zfill(width)
```

**DE Use Case:** Creating zero-padded IDs, batch numbers

In [ ]:
# zfill() - Create consistent ID formats
order_numbers = ["1", "42", "123", "9999"]

print("Creating 8-digit order IDs:")
print("-" * 30)
for num in order_numbers:
    padded = num.zfill(8)
    print(f"'{num}' → '{padded}'")

In [ ]:
# zfill() handles negative numbers correctly
numbers = ["42", "-42", "7", "-7"]

print("zfill() with negative numbers:")
print("-" * 30)
for num in numbers:
    padded = num.zfill(5)
    print(f"'{num}' → '{padded}'")

## 6.2 `ljust()`, `rjust()`, `center()` - Text Alignment

**Syntax:**
```
string.ljust(width[, fillchar])   # Left-align
string.rjust(width[, fillchar])   # Right-align
string.center(width[, fillchar])  # Center
```

**DE Use Case:** Fixed-width file formats, formatted reports

In [ ]:
# Creating fixed-width formatted output
data = [
    ("John Doe", 1500.00, "NYC"),
    ("Jane Smith", 2500.50, "LA"),
    ("Bob Johnson", 750.25, "Chicago"),
]

print("Fixed-width formatted report:")
print("=" * 45)
# Header
print(f"{'Name'.ljust(15)} | {'Amount'.rjust(10)} | {'City'.center(10)}")
print("-" * 45)
# Data rows
for name, amount, city in data:
    print(f"{name.ljust(15)} | {str(amount).rjust(10)} | {city.center(10)}")

---
# Quick Reference: Most Used Methods in Data Engineering

| Method | Use Case |
|--------|----------|
| `strip()` | Clean whitespace from raw data |
| `lower()` | Normalize emails, case-insensitive matching |
| `split()` | Parse CSV/log data into fields |
| `join()` | Build CSV rows, file paths |
| `replace()` | Standardize formats, clean data |
| `startswith()`/`endswith()` | Filter files, identify record types |
| `isdigit()` | Validate numeric strings |
| `find()` | Locate substrings for parsing |
| `partition()` | Extract key-value pairs |